In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import bokeh as bk
%matplotlib inline

# Analysis of stations : Visualise missing data and sample data

In [ ]:
df_stations = pd.read_csv('exercise_data/example_sprit_cut_station.csv',sep=';')

# Visualise number of missing entries
msno.matrix(df_stations);

In [ ]:
df_stations.sample(50)

# Analysis of stations : Clean data

We observe that there are irrelevant columns and duplicates. Moreover, invalid values need to be dealt with.

In [ ]:
cols_to_drop = ['VERSION','VERSION_TIME','HOUSE_NUMBER','PUBLIC_HOLIDAY_IDENTIFIER']
# Drop irrelevant columns
df_stations.drop(cols_to_drop,inplace=True,axis=1)

In [ ]:
# Drop duplicates
df_stations.drop_duplicates(inplace=True)

In [ ]:
# Investigate about the null data
null_data = df_stations[df_stations.isnull().any(axis=1)]
null_data

In [ ]:
# Replace Invalid Streets and Places
df_stations.update(df_stations[['STREET','PLACE']].fillna('Unknown'))

In [ ]:
# Replace invalid entries in postcode
df_stations['POST_CODE'].fillna(value=0, inplace=True)
df_stations['POST_CODE'].replace(to_replace=['\\N'],value='0',inplace=True)
df_stations['POST_CODE'].replace(to_replace=['nicht'],value='0',inplace=True)

In [ ]:
# Clean invalid brands
inactive_brands=df_stations[df_stations['BRAND']=='nicht mehr aktiv']
df_stations.drop(inactive_brands.index,axis=0,inplace=True)

In [ ]:
# Replace NaN s and \\N s in brands
df_stations['BRAND'].fillna(value='No Brand', inplace= True)
df_stations['BRAND'].replace(to_replace=['\\N'],value='No Brand',inplace=True)

In [ ]:
# Put brands and names in title case
df_stations['BRAND'] = df_stations['BRAND'].str.title()
df_stations['NAME'] = df_stations['NAME'].str.title()
#Visualise unique brands
sorted(df_stations.BRAND.unique())

In [ ]:
# More brand cleaning
df_stations.replace({'BRAND' : { '^Auto Zotz.*' : 'Auto Zotz', 
                                           '^Frei.*' : 'Freie Tankstelle',
                                           '^Raiffeisen.*' : 'Raiffeisen Tankstelle',
                                           '^Sb.*' : 'Sb Markt Tankstelle',
                                           '^Supermarkt.*' : 'Supermarkt Tankstelle'
                                            }},regex=True,inplace=True)
#Visualise unique brands
sorted(df_stations.BRAND.unique())

In [ ]:
# 'No name' brand can be replaced by names
df_stations['BRAND']=np.where(df_stations['BRAND']== 'No Brand', df_stations['NAME'], df_stations['BRAND'])

#Visualise unique brands
sorted(df_stations.BRAND.unique())

In [ ]:
# Even more brand cleaning
df_stations.replace({'BRAND' : { '^Autohaus Holz.*' : 'Autohaus Holz', 
                                           '^Eberhardt.*' : 'Eberhardt',
                                           '^Globus Handelshof .*' : 'Globus Handelshof Gmbh & Co. Kg',
                                           '^Sbk .*' : 'Sbk - Tankstelle',
                                            }},regex=True,inplace=True)
#Visualise unique brands
sorted(df_stations.BRAND.unique())

In [ ]:
# Visualise number of missing entries
msno.matrix(df_stations);

# Analysis of prices: Visualise missing data and sample data

In [ ]:
df_prices = pd.read_csv('exercise_data/example_sprit_cut_prices.csv',sep=';')
# Visualise number of missing entries
msno.matrix(df_prices);

In [ ]:
df_prices.sample(50)

# Analysis of prices : Clean data
We see that there are invalid prices along with duplicates. We need to normalise the dataset firstly and then perform the analysis

In [ ]:
# Drop duplicates
df_prices.drop_duplicates(inplace=True)
msno.matrix(df_prices);

In [ ]:
e5_invalid=df_prices[df_prices['E5']<=0]
e5_invalid

In [ ]:
# Remove invalid prices from the data
def remove_invalid_prices(fuel_name):
    fuel_invalid=df_prices[df_prices[fuel_name]<=0]
    df_prices.drop(fuel_invalid.index,axis=0,inplace=True)

remove_invalid_prices('E5')
remove_invalid_prices('E10')
remove_invalid_prices('DIESEL')

In [ ]:
df_prices.describe()

We see that the max, min are quite unrealistically bad. This needs normalisation

In [ ]:
def normalise_data(fuel_name):
    data_mean, data_std = df_prices[fuel_name].mean(), df_prices[fuel_name].std()
    # identify outliers upto 3 standard deviations
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off

    df_prices[fuel_name]=np.where(np.logical_or(df_prices[fuel_name] <
                                                lower,df_prices[fuel_name] > upper), 
                                  df_prices[fuel_name].median(), 
                                  df_prices[fuel_name])

normalise_data('E5')
normalise_data('E10')
normalise_data('DIESEL')

In [ ]:
df_prices.describe()